## Data_Modeling_Data_Construction 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

### Load and Process Data

In [2]:
bucket = '
'
file = 'WC_DeEscl_Data_Build_wClstr_v2_txtcln.csv'

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

/mnt/miniconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (86,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data shape:  (544529, 103)


In [3]:
list(df.columns.values)

['Year',
 'Claim_ID',
 'Claim_Num',
 'Claim_System_Entry_Dte',
 'Report_dte',
 'Injury_Dte',
 'Claim_Status_Cde',
 'Current_PMT_Cde',
 'Current_Controverted_I',
 'current_total_incurred',
 'Indemnity_Incurred',
 'Medical_Incurred',
 'Expense_Incurred',
 'Account_Name',
 'Top Account Name',
 'Lost_Time_Cde',
 'Jurisdiction_State_Cde',
 'Intake_Method',
 'Primary_Nature_Of_Injury_Cde',
 'Secondary_Nature_Of_Injury_Cde',
 'Primary_Part_Of_Body_Cde',
 'Secondary_Part_Of_Body_Cde',
 'Initial_Trtm_Cde',
 'Occurrence_Cde',
 'Claimant_Age_Qty',
 'Gender_Cde',
 'Tenure',
 'Report_Lag',
 'Avg_Wkly_Wge_Amt',
 'Runoff/Not Runoff',
 'Cust_Postal_Cde',
 'Claimant_Postal_Cde',
 'Occupation_Txt',
 'Accident_Note_Text',
 'Injury_Note_Text',
 'NAICS_Cde',
 'Segment_Today',
 'Dataset',
 'CRC_Rating_Plan_Code',
 'Initial_Seg',
 'InjurytoReport',
 'ReporttoEntry',
 'Prim_POB_Category_Cde',
 'Prim_POB_Dsc',
 'Prim_POB_Category_Dsc',
 'Scd_POB_Category_Cde',
 'Scd_POB_Dsc',
 'Scd_POB_Category_Dsc',
 'Occur_D

In [5]:
df['Acc_Txt_Clned'].head(10)

0       assoicate injure restriction say keyboard drop
1    work lightbulb aisle rochelle notice ache thum...
2                             causeunspecified medical
3    walk say couldnt see walk eye station wash eye th
4                    josh take item microwave fell arm
5    drop home depot knife break room bend pick kni...
6                                              unknown
7                                state feel pain wrist
8                                        jerk arm hard
9                           felt light head faint lift
Name: Acc_Txt_Clned, dtype: object

In [6]:
df['Inj_Txt_Clned'].head(10)

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Inj_Txt_Clned, dtype: object

In [7]:
# check NA found in cleaned text
df[df['Acc_Txt_Clned'].isna()][['Accident_Note_Text']] 

Accident_Note_Text
16353                SAME
40485                  AS
47850           ASSOCIATE
49519            AS ABOVE
142134               SAME
171398                EMP
208775              OTHER
257390      SAME AS ABOVE
277563           EMPLOYEE
280447      SAME AS ABOVE
303416           EMPLOYEE
371668               SAME
411829     FROM ASSOCIATE
428917           EMPLOYEE
472893                 IW
476568           EMPLOYEE
501718      SAME AS ABOVE

In [8]:
# check NA found in cleaned text
df[df['Inj_Txt_Clned'].isna()][['Injury_Note_Text']] 

Injury_Note_Text
2021              OTHER
11103             OTHER
12285     SAME AS ABOVE
15328             OTHER
20662             OTHER
23585             OTHER
24862             OTHER
37490                NO
37556     SAME AS ABOVE
54470     SAME AS ABOVE
55685             OTHER
56869             OTHER
60432     SAME AS ABOVE
60437             OTHER
62297             OTHER
63383             OTHER
63789             OTHER
64837             OTHER
65943             OTHER
66316             OTHER
68131             OTHER
68511             OTHER
69254             OTHER
70961             OTHER
72384             OTHER
73123             OTHER
73180             OTHER
75746             OTHER
76836             OTHER
79465             OTHER
...                 ...
277563         EMPLOYEE
296775            OTHER
297539            OTHER
303232            OTHER
315245    SAME AS ABOVE
341027             LEFT
343491            OTHER
345625            RIGHT
347414    SAME AS ABOVE
347937            OTHER
352448    SAME AS ABOVE
354163            OTHER
354499            OTHER
359673             SAME
364153    SAME AS ABOVE
370270            OTHER
371171    SAME AS ABOVE
371748         EMPLOYEE
375274    SAME AS ABOVE
377305            OTHER
396627            OTHER
401307        ALL OTHER
452899            OTHER
458586            OTHER
460639         EMPLOYEE
506919    SAME AS ABOVE
509670               EE
526525               EE
538404            OTHER
538850               EE

[128 rows x 1 columns]

In [9]:
for i in range(df.shape[0]):
    if df.Accident_Note_Text[i] in {'SAME', 'AS ABOVE', 'SAME AS ABOVE'}:
        print(i)
        print(df[['Claim_ID', 'Claim_Num', 'Report_dte', 'Account_Name', 'Accident_Note_Text', 'Injury_Note_Text']].iloc[(i-1):(i+1), :])

16353
        Claim_ID  Claim_Num  Report_dte              Account_Name  \
16352  272626606  555C09600  2013-08-17  DARDEN RESTAURANTS, INC.   
16353  272630106  80DC03852  2013-08-28     ELI LILLY AND COMPANY   

                                      Accident_Note_Text Injury_Note_Text  
16352  JAIME WAS LIFTING A BOX OF SALAD MIX TO PUT IN...              NaN  
16353                                               SAME              NaN  
49519
        Claim_ID  Claim_Num  Report_dte            Account_Name  \
49518  822569806  413A75403  2013-03-12  OFFICEMAX INCORPORATED   
49519  822570406  390C00449  2013-03-14   ELI LILLY AND COMPANY   

                                      Accident_Note_Text Injury_Note_Text  
49518  ASSOCIATE WAS WALKING OUT FROM TRUCK AND TURNE...              NaN  
49519                                           AS ABOVE              NaN  
142134
         Claim_ID  Claim_Num  Report_dte                  Account_Name  \
142133  342746806  413C11564  2014-06-06 

In [10]:
for i in range(df.shape[0]):
    if df.Injury_Note_Text[i] in {'SAME', 'AS ABOVE', 'SAME AS ABOVE'}:
        print(i)
        print(df[['Claim_ID', 'Claim_Num', 'Report_dte', 'Account_Name', 'Accident_Note_Text', 'Injury_Note_Text']].iloc[(i-1):(i+1), :])

12285
        Claim_ID  Claim_Num  Report_dte              Account_Name  \
12284  202772506  80DC38031  2014-08-17        THE HOME DEPOT INC   
12285  202772606  390C60110  2014-08-17  DARDEN RESTAURANTS, INC.   

                                      Accident_Note_Text Injury_Note_Text  
12284  JABARI GOT INTO A FIGHT WITH ANOTHER ASSOCIATE...  BRUISECONTUSION  
12285  EMPLOYEE STATES HE SLIPPED ON A PEPPRONCINI AN...    SAME AS ABOVE  
37556
        Claim_ID  Claim_Num  Report_dte              Account_Name  \
37555  622785306  949C45088  2014-09-23        DEAN FOODS COMPANY   
37556  622787106  80DC42148  2014-09-29  DARDEN RESTAURANTS, INC.   

                                      Accident_Note_Text  \
37555         WALKING UP STAIRS SLIPPED AND TWISTED KNEE   
37556  DOUG WAS CUTTING STEAKS BAG AND SLIPPED CUTTIN...   

                                        Injury_Note_Text  
37555  WALKING UP STAIRS NOT USING HANDRAIL SLIPPED A...  
37556                                      SA

**Comments:** For the claims have notes indicating "same as above", the order of the reported claims may have been shifted. There is not enough evidence suggesting those fields can be filled by the content from the previous line. They will be marked as missing.

In [11]:
# fill NA as missing
df['Acc_Txt_Clned'] = df['Acc_Txt_Clned'].fillna('unknown')
df['Inj_Txt_Clned'] = df['Inj_Txt_Clned'].fillna('unknown')

In [12]:
# creating the feature matrix 
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start_time = time.time()

acct_vectorizer1 = TfidfVectorizer(max_features=1000)
inj_vectorizer1 = TfidfVectorizer(max_features=1000)
acct_tfidf1 = acct_vectorizer1.fit_transform(df['Acc_Txt_Clned']).toarray()
inj_tfidf1 = inj_vectorizer1.fit_transform(df['Inj_Txt_Clned']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 14.168870449066162 seconds ---


In [13]:
start_time = time.time()

acct_vectorizer2 = TfidfVectorizer(max_features=1000,
                                  ngram_range=(2, 2)) # look at two gram phrase only
inj_vectorizer2 = TfidfVectorizer(max_features=1000,
                                 ngram_range=(2, 2))
acct_tfidf2 = acct_vectorizer2.fit_transform(df['Acc_Txt_Clned']).toarray()
inj_tfidf2 = inj_vectorizer2.fit_transform(df['Inj_Txt_Clned']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 19.516467809677124 seconds ---


In [14]:
acct_features1 = acct_vectorizer1.get_feature_names()
acct_features2 = acct_vectorizer2.get_feature_names()

inj_features1 = inj_vectorizer1.get_feature_names()
inj_features2 = inj_vectorizer2.get_feature_names()

### Create Additional Features

In [15]:
# add parking lot indicator and interaction
df['prklot_idx'] = np.where(acct_tfidf2[:,acct_features2.index('park lot')] > 0, 1, 0)
df['prklot_strain_sprain'] = np.where((df['prklot_idx'] == 1 & 
                                     ((df['Prim_NOI_Dsc'] == 'Strain') | 
                                      (df['Prim_NOI_Dsc'] == 'Sprain'))), 
                                    1,0)
df['prklot_head_category'] = np.where((df['prklot_idx'] == 1) & 
                                      (df['Prim_POB_Category_Dsc'] == 'HEAD') , 
                                      1,0)
df['prklot_knee'] = np.where((df['prklot_idx'] == 1) & 
                            (df['Prim_POB_Dsc'] == 'Knee') , 
                            1,0)

In [16]:
# add pallet indicator and interaction
df['pallet_idx'] = np.where((acct_tfidf1[:, acct_features1.index('pallet')] > 0) |
                             (inj_tfidf1[:, inj_features1.index('pallet')] > 0), 1, 0)
df['pallet_lowerEx'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Category_Dsc'] == 'LOWER EXTREMITIES') , 
                                1,0)
df['pallet_head_category'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Category_Dsc'] == 'HEAD') , 
                                1,0)
df['pallet_lowerbck'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Dsc'] == 'Low back (lumbar or lumbo-sacral area)') , 
                                1,0)
df['pallet_handarea'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Dsc'].isin(['Finger(s)', 'Wrist', 'Hand', 'Thumb'])) , 
                                1,0)

In [17]:
df['allrg_idx'] = np.where((acct_tfidf1[:,acct_features1.index('allergic')] > 0) |
                           (inj_tfidf1[:,inj_features1.index('allergic')] > 0), 1, 0)
df['notfeelwell_idx'] = np.where(acct_tfidf2[:,acct_features2.index('feel well')] > 0, 1, 0)

In [18]:
df['szre_stroke_idx'] = np.where(
                        (inj_tfidf1[:,inj_features1.index('strokeseizure')] > 0) |
                        (inj_tfidf1[:,inj_features1.index('seizure')] > 0) |
                        (inj_tfidf1[:,inj_features1.index('stroke')] > 0)
                        , 1, 0)

In [16]:
# def seg_analysis(df, target, seg, group,  metrics):
#     """
#         input: 
#         df = dataframe;
#         target = response var, a list;
#         seg = segmentation var, a string;
#         group = group by criteria, a list;
#         metrics = aggregation metrics, a list 
        
#         return: summary dataframe
#     """
# #     seg = list(seg)
#     target = list(target)
#     group = list(group)
    
#     col = target + group
    
#     tb1 = df[col].groupby(group).agg(metrics)
# #     tb2 = df[col].groupby(group).agg(metrics)
#     tb2 = df[df[seg] == 1][col].groupby(group).agg(metrics)
    
#     summary = pd.merge(tb1, tb2, on = group, how = 'outer')
#     return summary

In [43]:
# df = df.drop(columns=['szre_idx'])

In [28]:
# seg_analysis(df, 
#              ['deEscInd3000'], 
#              'notfeelwell_idx', 
#              ['Prim_POB_Dsc'],
#              ['count', 'mean'])

deEscInd3000_x            \
                                                count      mean   
Prim_POB_Dsc                                                      
Abdomen                                         14297  0.643492   
Ankle                                           21831  0.701754   
Artificial Appliance                               59  0.898305   
Body Systems and Multiple Body Systems           1618  0.803461   
Brain                                            7005  0.861956   
Buttocks                                         2612  0.813936   
Chest, ribs or sternum                           9030  0.787375   
Disc in low or middle back                        272  0.496324   
Disc in neck                                       70  0.485714   
Ear(s)                                           4478  0.638901   
Elbow                                           11159  0.675060   
Eye(s)                                          12552  0.947100   
Facial bones                                      890  0.625843   
Facial soft tissue                               7349  0.851953   
Finger(s)                                       34230  0.808706   
Foot                                            21218  0.768593   
Great Toe                                        1939  0.847860   
Hand                                            31927  0.791399   
Head                                            34562  0.785834   
Heart                                            3371  0.933254   
Hip                                              5864  0.668656   
Insufficient Information                         5014  0.825888   
Internal organs                                   720  0.719444   
Knee                                            41048  0.632455   
Larynx                                            198  0.858586   
Low back (lumbar or lumbo-sacral area)          80770  0.696694   
Lower arm                                       17672  0.778293   
Lower extremities                                1299  0.646651   
Lower leg                                       13781  0.743633   
Lumbar and/or Sacral Vertebrae                    178  0.561798   
Lungs                                            6752  0.782879   
Mouth                                            1437  0.860125   
Multiple body parts                             14294  0.700014   
Neck                                            12570  0.644710   
Neck soft tissue                                  828  0.692029   
No Physical Injury                              14012  0.880816   
Nose                                             1930  0.795337   
Pelvis                                            868  0.550691   
Sacrum or coccyx                                  581  0.736661   
Shoulder                                        38111  0.568550   
Skull                                             253  0.711462   
Spinal cord (low or middle back area)             215  0.567442   
Spinal cord (neck area)                            64  0.640625   
Teeth                                            1306  0.696018   
Thigh                                            3223  0.739684   
Thumb                                           11248  0.824769   
Toe(s)                                           4020  0.863682   
Trachea                                           224  0.892857   
Trunk                                             852  0.733568   
Upper arm, clavicle or scapula                   5187  0.584924   
Upper back (thoracic area)                       6340  0.742587   
Upper extremities                                1905  0.645669   
Vertebrae in neck                                  74  0.540541   
Wrist                                           26294  0.716779   
Wrist(s) & Hand(s)                               4928  0.609781   

                                       deEscInd3000_y            
                                                count      mean  
Prim_POB_Dsc        

In [ ]:
# export raw modeldf 

filename = 'WC_DeEscl_Modeldf_v2.csv'
df.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)
#df = pd.read_csv('WC_DeEscl_Modeldf_v2.csv')

### Porcess and Encode Modeldf

In [ ]:
target_col = [
             'current_total_incurred',
             'Indemnity_Incurred',
             'Medical_Incurred',
             'Expense_Incurred',    
             'deEscInd3500',
             'deEscInd3000',
             'deEscInd2500',
             'deEscInd2000',
             'deEscInd1500',
             'deEscInd1000',
             'over100kFlg',
             'zeroLossFlg']

analysis_col = [
             'Current_PMT_Cde',
             'Current_Controverted_I',
             'CRC_Rating_Plan_Code']

watch_col = [
            'Account_Name',
            'Top Account Name',
            'Account_Grp2',
            'Cust_Postal_Cde',
            'Claimant_Postal_Cde',
            'Occupation_Txt',
            'Occupation_Grp',
            'Occupation_Grp2',
            'NAICS_Cde',
            'PrimNOI_Grp20',
            'ScdNOI_Grp10',
            'PrimPOB_Grp30',
            'ScdPOB_Grp10',
            'Occur_Grp30']

other_col = [ 
             'Acc_Txt_Clned',
             'Inj_Txt_Clned',
             'Claim_ID',
             'Claim_Num',
             'Claim_System_Entry_Dte',
             'Report_dte',
             'Injury_Dte',
             'Claim_Status_Cde',
             'Jurisdiction_State_Cde',
             'Intake_Method',
             'Primary_Nature_Of_Injury_Cde',
             'Secondary_Nature_Of_Injury_Cde',
             'Primary_Part_Of_Body_Cde',
             'Secondary_Part_Of_Body_Cde',
             'Occurrence_Cde',
             'Report_Lag',
             'Runoff/Not Runoff',
             'Accident_Note_Text',
             'Injury_Note_Text',
             'Segment_Today',
             'Dataset',
             'Prim_POB_Category_Cde',
             'Scd_POB_Category_Cde',
             'Region',
             'Claimant_Latitude',
             'Claimant_Longitude',
             'Cust_Latitude',
             'Cust_Longitude',
             'Initial_Seg']

model_col = [
             'Year',
             'Initial_Trtm_Cde',
             'Lost_Time_Cde',
             'Claimant_Age_Qty',
             'Gender_Cde',
             'Tenure',
             'Avg_Wkly_Wge_Amt',
             'InjurytoReport',
             'ReporttoEntry',

             'Prim_POB_Dsc',
             'Prim_POB_Category_Dsc',
             'Scd_POB_Dsc',
             'Scd_POB_Category_Dsc',
             'Occur_Dsc',
             'Prim_NOI_Dsc',
             'Scd_NOI_Dsc',
 
             'Dist_to_work',   
             'age_5grp',
             'age_8grp',
             'age_10grp',
             'Tenure_5grp',
             'Avg_Wkly_Wge_Amt_5grp',
             'InjurytoReport_5grp',
             'ReporttoEntry_5grp',

             'Occupation_Grp1',
             'Account_Grp1',
             'NAICS_Grp',
             'CRC_Rating_Plan_Code',
        
             'PrimPOB_km_2c',
             'PrimPOB_km_4c',
             'PrimPOB_pam_2c',
             'PrimPOB_pam_5c',
             'PrimNOI_km_3c',
             'PrimNOI_km_6c',
             'PrimNOI_km_12c',
             'PrimNOI_pam_6c',
             'PrimNOI_pam_7c',
             'PrimNOI_pam_9c',
             'Occur_km_3c',
             'Occur_km_4c',
             'Occur_km_6c',
             'Occur_pam_4c',
             'Occur_pam_9c',
             'prklot_idx',
             'prklot_strain_sprain',
             'prklot_head_category',
             'prklot_knee',
             'pallet_idx',
             'pallet_lowerEx',
             'pallet_head_category',
             'pallet_lowerbck',
             'pallet_handarea',
             'allrg_idx',
             'notfeelwell_idx',
             'szre_stroke_idx']

In [ ]:
moddf = df[model_col + ['folds']]
moddf.shape

In [ ]:
moddf.dtypes

In [ ]:
moddf[['PrimNOI_km_3c',             
        'PrimNOI_km_6c',
        'PrimNOI_km_12c',       
        'PrimNOI_pam_6c',       
        'PrimNOI_pam_7c',       
        'PrimNOI_pam_9c',
        'PrimPOB_km_2c',
        'PrimPOB_km_4c',
        'PrimPOB_pam_2c',
        'PrimPOB_pam_5c']] = moddf[['PrimNOI_km_3c',             
                                    'PrimNOI_km_6c',
                                    'PrimNOI_km_12c',       
                                    'PrimNOI_pam_6c',       
                                    'PrimNOI_pam_7c',       
                                    'PrimNOI_pam_9c',
                                   'PrimPOB_km_2c',
                                    'PrimPOB_km_4c',
                                    'PrimPOB_pam_2c',
                                    'PrimPOB_pam_5c']].astype(str)

In [ ]:
moddf.isnull().sum()

In [ ]:
#fill na with 'missing'
moddf['Dist_to_work_missing'] = np.where(moddf['Dist_to_work'].isnull(), 1, 0)
moddf['Dist_to_work']  = moddf['Dist_to_work'].fillna(0.0)

In [ ]:
moddf[['Dist_to_work_missing']].groupby(['Dist_to_work_missing']).size()

In [ ]:
adjcol = moddf.columns.values[np.where(moddf.isnull().sum() > 0)]
print(adjcol)
for col in adjcol:
    moddf[col] = moddf[col].fillna('missing')

In [ ]:
# sanity check
moddf.isnull().sum()

In [ ]:
#split data in train and test
X_train = moddf[moddf['folds'] <= 3].drop(['folds'], axis = 1)
X_test = moddf[moddf['folds'] > 3].drop(['folds'], axis = 1)

In [ ]:
# one-hot encoding categorical variable
import time
start_time = time.time()

catcol = X_train.select_dtypes(include=[object]).columns.values

X_train_onehot = X_train.copy()

for col in catcol:
    X_train_onehot = pd.get_dummies(X_train_onehot, 
                                    columns=[col], 
                                    prefix = [col])

print("--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
start_time = time.time()

catcol = X_test.select_dtypes(include=[object]).columns.values

X_test_onehot = X_test.copy()

for col in catcol:
    X_test_onehot = pd.get_dummies(X_test_onehot, 
                                    columns=[col], 
                                    prefix = [col])

print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
print(X_train_onehot.shape)
print(X_test_onehot.shape)

In [ ]:
# examine dimension difference in train and test
trainonly_col = set(list(X_train_onehot)) - set(list(X_test_onehot))
testonly_col = set(list(X_test_onehot)) - set(list(X_train_onehot))

In [ ]:
for col in trainonly_col:
    print(col, X_train_onehot[col].sum())

In [ ]:
for col in testonly_col:
    print(col, X_test_onehot[col].sum())

all of these columns have very low count, should be removed from data

In [ ]:
X_train_onehot = X_train_onehot.drop(trainonly_col, axis = 1)
X_test_onehot = X_test_onehot.drop(testonly_col, axis = 1)

In [ ]:
# sanity check
print(X_train_onehot.shape)
print(X_test_onehot.shape)

In [ ]:
# export train and test data
X_train_onehot.to_csv(path_or_buf= f's3://cortex-ni-wc-claims/WC_DeEscl_Data_Build_wClstr_v2_dtrain.csv', index=False)
X_test_onehot.to_csv(path_or_buf= f's3://cortex-ni-wc-claims/WC_DeEscl_Data_Build_wClstr_v2_dtest.csv', index=False)